In [1]:
import os 
from dotenv import load_dotenv
load_dotenv() ## laoding all the environment variables

groq_api_key = os.getenv("GROQ_API_KEY")


In [4]:
from langchain_groq import ChatGroq
model = ChatGroq(model = "Gemma2-9b-It",api_key=groq_api_key)
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000001E2F7A72410>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000001E2F7A737C0>, model_name='Gemma2-9b-It', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [5]:
from langchain_core.messages import HumanMessage
model.invoke([HumanMessage(content="Hi, My name is azhar and i am Gen AI engineer")])

AIMessage(content="Hi Azhar! It's nice to meet you.  \n\nThat's awesome! Being a Gen AI engineer is such a fascinating field. What kind of projects are you working on? \n\nI'm always eager to learn more about the work people are doing with AI. 😊\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 62, 'prompt_tokens': 22, 'total_tokens': 84, 'completion_time': 0.112727273, 'prompt_time': 0.002120865, 'queue_time': 0.018609724, 'total_time': 0.114848138}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-7e5848b4-5842-4ca7-b0ac-292d699976a2-0', usage_metadata={'input_tokens': 22, 'output_tokens': 62, 'total_tokens': 84})

In [6]:
from langchain_core.messages import AIMessage
model.invoke(
    [
        HumanMessage(content="Hi, My name is azhar and i am Gen AI engineer"),
        AIMessage(content="Hi Azhar! It's nice to meet you.  \n\nThat's awesome! Being a Gen AI engineer is such a fascinating field. What kind of projects are you working on? \n\nI'm always eager to learn more about the work people are doing with AI. 😊\n"),
        HumanMessage(content="Hey whats my name and wat do I do?")
    ]
)

AIMessage(content="You said your name is Azhar, and that you are a Gen AI engineer! \n\nIs there anything else you'd like to tell me about yourself or your work? 😄  \n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 41, 'prompt_tokens': 102, 'total_tokens': 143, 'completion_time': 0.074545455, 'prompt_time': 0.005100459, 'queue_time': 0.016175630000000003, 'total_time': 0.079645914}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-de16b2f4-b7fa-4dfa-a240-4f67b4b92f93-0', usage_metadata={'input_tokens': 102, 'output_tokens': 41, 'total_tokens': 143})

In [9]:
### message history
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory

store = {}
def get_session_history(session_id:str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory() 
    return store[session_id]

with_message_history = RunnableWithMessageHistory(model,get_session_history)


In [10]:
config = {"configurable":{"session_id":"chat1"}}

In [13]:
response = with_message_history.invoke(
    [HumanMessage(content="Hi, My name is azhar and i am Gen AI engineer")],
    config=config
)

In [14]:
response.content

"Hi Azhar! It's nice to meet you. \n\nBeing a Gen AI engineer is a fascinating field. What kind of projects are you working on these days?  Are you focused on text generation, image synthesis, something else entirely? I'm always eager to learn more about the innovative work being done in this space.  😊 \n\n"

In [15]:
with_message_history.invoke(
    [HumanMessage(content="what is my name?")],
    config=config
)

AIMessage(content='Your name is Azhar.  😊 \n\nI remembered from our first introduction!  \n\n\n\n\n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 198, 'total_tokens': 220, 'completion_time': 0.04, 'prompt_time': 0.0120653, 'queue_time': 0.180608758, 'total_time': 0.0520653}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-09889c33-3e7e-443a-9f9b-97323b8509d6-0', usage_metadata={'input_tokens': 198, 'output_tokens': 22, 'total_tokens': 220})

In [16]:
## change the config -->session
config1 = {"configurable":{"session_id":"chat2"}}
response = with_message_history.invoke(
    [HumanMessage(content="what is my name?")],
    config=config1
)
print(response.content)

As an AI, I have no access to personal information about you, including your name. If you'd like to tell me your name, I'd be happy to know!



In [17]:
response = with_message_history.invoke(
    [HumanMessage(content="my name is afridi")],
    config=config1
)
response.content

'Nice to meet you, Afridi! 👋 \n\nHow can I help you today? 😊  \n'

In [18]:
response = with_message_history.invoke(
    [HumanMessage(content="what is my name?")],
    config=config1
)
print(response.content)

Your name is Afridi. 😊  

I remember!  

Is there anything else I can help you with?  



### Prompt Templates
working with prompt templates

In [21]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system", 
            "You are a helpful assistant.Answer all the question to the nest of your ability"
            ),
        MessagesPlaceholder(variable_name="messages")
    ]

)
chain = prompt | model

In [24]:
chain.invoke({"messages":[HumanMessage(content="Hey my name is azhar")]})

AIMessage(content="Hi Azhar, it's nice to meet you!\n\nI'm ready to answer your questions to the best of my ability. What can I help you with today? 😊  \n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 41, 'prompt_tokens': 31, 'total_tokens': 72, 'completion_time': 0.074545455, 'prompt_time': 0.002788413, 'queue_time': 0.020458625, 'total_time': 0.077333868}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-b4cb97f4-1f03-4720-bac2-fa1edb23a991-0', usage_metadata={'input_tokens': 31, 'output_tokens': 41, 'total_tokens': 72})

In [25]:
with_message_history = RunnableWithMessageHistory(chain, get_session_history)


In [26]:
config = {"configurable":{"session_id":"chat3"}}
response = with_message_history.invoke([HumanMessage(content="Hey my name is azhar")],
                                        config=config
                                        )
response.content

"Hello Azhar! It's nice to meet you.\n\nHow can I help you today? 😊  \n\nI'm ready for any questions you have!  \n\n"

In [36]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system", 
            "You are a helpful assistant.Answer all the question to the nest of your ability"
            ),
        MessagesPlaceholder(variable_name="messages"),
    ]

)
chain = prompt | model

In [37]:
response = chain.invoke({"messages":[HumanMessage(content="Hi my name is azhar")], "language":"Hindi"})
response.content

"Hello Azhar, it's nice to meet you!\n\nI'm happy to help with any questions you have. Just ask away!  😊 \n"

In [38]:
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages"
)

In [41]:
config = {"configurable":{"session_id":"chat4"}}
response = with_message_history.invoke(
    {'messages': [HumanMessage(content="Hi, i am azhar")], "language": "english"},
    config=config
)
response.content


"Hi Azhar!  \n\nIt's nice to meet you.  \n\nWhat can I do for you today? 😊  \n"

In [42]:
response = with_message_history.invoke(
    {'messages': [HumanMessage(content="what's my name?")], "language": "english"},
    config=config
)
response.content


'Your name is Azhar. 😊 \n\nHow can I help you further?  \n'

### Manage the conversation History

In [54]:
from langchain_core.messages import SystemMessage, trim_messages

trimmer = trim_messages(
    max_tokens = 45,
    strategy = "last", 
    token_counter = model, 
    include_system = True,
    allow_partial = False,
    start_on = "human"
)
messages = [
    SystemMessage(content="You're a good assistant"),
    HumanMessage(content="hi! i am afridi"),
    AIMessage(content="Hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="what's 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem"),
    HumanMessage(content="having Fun?"),
    AIMessage(content="yes!"),
]
trimmer.invoke(messages)


[SystemMessage(content="You're a good assistant", additional_kwargs={}, response_metadata={}),
 HumanMessage(content='I like vanilla ice cream', additional_kwargs={}, response_metadata={}),
 AIMessage(content='nice', additional_kwargs={}, response_metadata={}),
 HumanMessage(content="what's 2 + 2", additional_kwargs={}, response_metadata={}),
 AIMessage(content='4', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='thanks', additional_kwargs={}, response_metadata={}),
 AIMessage(content='no problem', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='having Fun?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='yes!', additional_kwargs={}, response_metadata={})]

In [55]:
from operator import itemgetter

from langchain_core.runnables import RunnablePassthrough

chain=(
    RunnablePassthrough.assign(messages= itemgetter("messages")|trimmer)
    |prompt
    |model
)
response = chain.invoke(
    {
    "messages": messages + [HumanMessage(content="what ice cream do i like")],
    "language":"English"
    }
)
response.content

"As a large language model, I have no memory of past conversations or personal information about you, including your ice cream preferences. \n\nWhat's your favorite flavor? 😊🍦\n"

In [57]:
response = chain.invoke(
    {
    "messages": messages + [HumanMessage(content="what problem do i ask for above?")],
    "language":"English"
    }
)
response.content

'You asked me: "What\'s 2 + 2?" \n\n\nLet me know if you have any other questions! I\'m here to help. 😊\n'

In [58]:
## lets wrap this in the Message history
with_message_history = RunnableWithMessageHistory(
    chain, 
    get_session_history,
    input_messages_key="messages",
)
config = {"configurable": {"session_id":"chat5"}}

In [60]:
response = with_message_history.invoke(
    {
        "messages": messages + [HumanMessage(content="what's my name?")],
        "language": "English",
    },
    config = config,
)
response.content

"As an AI, I have no memory of past conversations and don't know your name.  What would you like me to call you? 😊  \n"